<h1>Build an Interactive Dashboard with Ploty Dash</h1>

<img src="dash_completed.png">

## Import Dash from pandas

In [1]:
!pip3 install pandas dash

## Load data

In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2022-04-22 00:01:50--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2,4K) [text/csv]
Saving to: 'spacex_launch_dash.csv'

     0K ..                                                    100%  562M=0s

2022-04-22 00:01:51 (562 MB/s) - 'spacex_launch_dash.csv' saved [2476/2476]



In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

--2022-04-22 00:01:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2110 (2,1K) [text/x-python]
Saving to: 'spacex_dash_app.py'

     0K ..                                                    100%  419M=0s

2022-04-22 00:01:53 (419 MB/s) - 'spacex_dash_app.py' saved [2110/2110]



<h3>Test the skeleton app by running the jupyter notebook "run spacex_dash_app"</h3>

You should see a nearly blank web page indicating a successfully running dash app.
Next, let's fill the skeleton app with required input/output components and callback functions.

If you need to refresh your memory about Plotly Dash components and callback functions, you may refer to the lab you have learned before:

<h2>Import Libraries</h2>

In [4]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\gustavo\AppData\Local\Temp/ipykernel_10536/1855361594.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\gustavo\AppData\Local\Temp/ipykernel_10536/1855361594.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [5]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [6]:
max_payload

9600.0

In [7]:
min_payload

0.0

In [8]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


<h3>Launch sites</h3>

In [9]:
df_launchSites = spacex_df['Launch Site'].unique()

In [10]:
for i in df_launchSites:
    print(i)

CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [11]:
# Create a dash application
app = dash.Dash(__name__)

In [12]:
# Create an app layout
app.layout = html.Div(
    children=[
        html.H1(
            'SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}
        ),
        # TASK 1: Add a dropdown list to enable Launch Site selection
        # The default select value is for ALL sites

        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'},
                {'label': df_launchSites[0], 'value': df_launchSites[0]},
                {'label': df_launchSites[1], 'value': df_launchSites[1]},
                {'label': df_launchSites[2], 'value': df_launchSites[2]},
                {'label': df_launchSites[3], 'value': df_launchSites[3]},
            ],
            value='ALL',
            placeholder='Select a Launch Site here',
            searchable=True
        ),
        
        html.Br(),

        # TASK 2: Add a pie chart to show the total successful launches count for all sites
        # If a specific launch site was selected, show the Success vs. Failed counts for the site
        html.Div(
            dcc.Graph(
                id='success-pie-chart'
            )
        ),
        html.Br(),

        html.P("Payload range (Kg):"),
        # TASK 3: Add a slider to select payload range
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=10000,
            step=500,
            value=[
                min_payload, 
                max_payload
            ]
        ),

        # TASK 4: Add a scatter chart to show the correlation between payload and launch success
        html.Div(
            dcc.Graph(
                id='success-payload-scatter-chart'
            )
        ),
    ]
)


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),  # saida pada o gráfico de id='success-pie-chart'
    Input(component_id='site-dropdown', component_property='value')         # a entrada vem da lista de seleção id='site-dropdown' com o value lá escolhido
)
def get_pie_chart(selected_site):
    filtered_df = spacex_df
    if selected_site == 'ALL':  # Escolheu ALL
        fig = px.pie(
            filtered_df, values='class',
            names='Launch Site',
            title='Success Count for all launch sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']== selected_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(
            filtered_df,
            values='class count',
            names='class',
            title=f"Total Success Launches for site {selected_site}")
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart',component_property='figure'),
    [Input(component_id='site-dropdown',component_property='value'), Input(component_id='payload-slider',component_property='value')]
)

def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success count on Payload mass for site {entered_site}")
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server(host='localhost', port = 7030)


Dash is running on http://localhost:7030/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:7030/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2022 13:39:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:39:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:39:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:39:38] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:39:38] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:39:38] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:39:38] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:39:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:39:39] "POST /_dash-update-component HTTP/1.1" 200 -


<h2>App complete</h2>

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

df_launchSites = spacex_df['Launch Site'].unique()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(
    children=[
        html.H1(
            'SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}
        ),
        # TASK 1: Add a dropdown list to enable Launch Site selection
        # The default select value is for ALL sites

        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'},
                {'label': df_launchSites[0], 'value': df_launchSites[0]},
                {'label': df_launchSites[1], 'value': df_launchSites[1]},
                {'label': df_launchSites[2], 'value': df_launchSites[2]},
                {'label': df_launchSites[3], 'value': df_launchSites[3]},
            ],
            value='ALL',
            placeholder='Select a Launch Site here',
            searchable=True
        ),
        
        html.Br(),

        # TASK 2: Add a pie chart to show the total successful launches count for all sites
        # If a specific launch site was selected, show the Success vs. Failed counts for the site
        html.Div(
            dcc.Graph(
                id='success-pie-chart'
            )
        ),
        html.Br(),

        html.P("Payload range (Kg):"),
        # TASK 3: Add a slider to select payload range
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=10000,
            step=500,
            value=[
                min_payload, 
                max_payload
            ]
        ),

        # TASK 4: Add a scatter chart to show the correlation between payload and launch success
        html.Div(
            dcc.Graph(
                id='success-payload-scatter-chart'
            )
        ),
    ]
)


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),  # saida pada o gráfico de id='success-pie-chart'
    Input(component_id='site-dropdown', component_property='value')         # a entrada vem da lista de seleção id='site-dropdown' com o value lá escolhido
)
def get_pie_chart(selected_site):
    filtered_df = spacex_df
    if selected_site == 'ALL':  # Escolheu ALL
        fig = px.pie(
            filtered_df, values='class',
            names='Launch Site',
            title='Success Count for all launch sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']== selected_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(
            filtered_df,
            values='class count',
            names='class',
            title=f"Total Success Launches for site {selected_site}")
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart',component_property='figure'),
    [Input(component_id='site-dropdown',component_property='value'), Input(component_id='payload-slider',component_property='value')]
)

def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success count on Payload mass for site {entered_site}")
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server(host='localhost', port = 7030)


Dash is running on http://localhost:7030/

Dash is running on http://localhost:7030/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:7030/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2022 13:41:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:41:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:41:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:41:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:41:07] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:41:07] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:41:07] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:41:07] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2022 13:41:07] "POST /_dash-update-component HTTP/1.1" 200 -
